# Insiders - All In One Place

## The All In One Place company

The All in One Place company is a Multibrand Outlet company. It sells second-line products of several brands at a lower price, through an e-commerce.

In just over 1 year of operation, the marketing team realized that some customers in its base buy more expensive products, with high frequency and end up contributing a significant portion of the company's revenue.

Based on this perception, the marketing team will launch a loyalty program for the best customers in the base, called Insiders. But the team does not have an advanced knowledge of data analysis to elect program participants.

For this reason, the marketing team asked the data team to select eligible customers for the program, using advanced data manipulation techniques.

---

## Project Objectives

You are part of All In One Place's team of data scientists who need to determine who are the eligible customers to be part of Insiders. In possession of this list, the Marketing team will carry out a sequence of personalized and exclusive actions to the group, in order to increase sales and purchase frequency.

As a result of this project, you are expected to submit a list of people eligible to participate in the Insiders program, along with a report answering the following questions:

1. Who are the people eligible to participate in the Insiders program?
2. How many customers will be part of the group?
3. What are the main characteristics of these customers?
4. What is the percentage of revenue contribution, coming from Insiders?
5. What is this group's revenue expectation for the coming months?
6. What are the conditions for a person to be eligible for Insiders?
7. What are the conditions for a person to be removed from Insiders?
8. What is the guarantee that the Insiders program is better than the rest of the base?
9. What actions can the marketing team take to increase revenue?

---

## Data 

The dataset is available on the [Kaggle platform](https://www.kaggle.com/vik2012kvs/high-value-customers-identification).

Each line represents a sale transaction, which took place between the period of November 2016 and December 2017.

The dataset includes the following information:
* InvoiceNo: Invoice number (A 6-digit integral number uniquely assigned to each transaction)
* StockCode: Product (item) code
* Description: Product (item) name
* Quantity: The quantities of each product (item) per transaction
* InvoiceDate: The day when each transaction was generated
* UnitPrice: Unit price (Product price per unit)
* CustomerID: Customer number (Unique ID assigned to each customer)
* Country: Country name (The name of the country where each customer resides)

# Summary
* [1. Invoices Dataframe](#1.)
    * [1.1 Missing Values](#1.1)
    * [1.2 New Features](#1.2)
    * [1.3 Negative Quantities](#1.3)
    * [1.4 Data Analysis](#1.4)
* [2. Customers Dataframe](#2.)
    * [2.1 Dataframe](#2.1)
    * [2.2 New Features](#2.2)
    * [2.3 Data Analysis](#2.3)
    * [2.4 Data Preprocessing](#2.4)
* [3. Model](#3.)
    * [3.1 K-Means](#3.1)
    * [3.2 Agglomerative Clustering](#3.2)
    * [3.3 DBSCAN](#3.3)
* [4. Conclusion](#4.)
    * [4.1 Who are the people eligible to participate in the Insiders program?](#4.1)
    * [4.2 How many customers will be part of the group?](#4.2)
    * [4.3 What are the main characteristics of these customers?](#4.3)
    * [4.4 What is the percentage of revenue contribution, coming from Insiders?](#4.4)
    * [4.5 What is this group's revenue expectation for the coming months?](#4.5)
    * [4.6 What are the conditions for a person to be eligible for Insiders?](#4.6)
    * [4.7 What are the conditions for a person to be removed from Insiders?](#4.7)
    * [4.8 What is the guarantee that the Insiders program is better than the rest of the base?](#4.8)
    * [4.9 What actions can the marketing team take to increase revenue?](#4.9)

# References
* [Targeting your most valuable customers in a post-Pareto world](https://www.dynamicyield.com/lesson/targeting-high-value-customers/)
* [What Is a High-Value Customer? [+5 Ways to Identify Them]](https://blog.hubspot.com/service/high-value-customer)

# Import the python libraries

In [1]:
# data analysis
import pandas as pd
import numpy as np

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick

# machine learning
import sklearn.cluster as cluster
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import ward, dendrogram
from sklearn.neighbors import NearestCentroid
from scipy.spatial import distance

ModuleNotFoundError: No module named 'seaborn'

# 1. Invoices Dataframe <a class='anchor' id='1.'></a>

In [ ]:
df_inv = pd.read_csv('csv/Ecommerce.csv', encoding='ISO-8859-1')
df_inv.head(10)

In [ ]:
df_inv.info()

In [ ]:
df_inv.describe(include='all')

## 1.1 Missing values <a class='anchor' id='1.1'></a>

In [ ]:
print(f'Only features contained missing value in Training Dataset')
temp = df_inv.isnull().sum()
print(temp.loc[temp!=0], '\n')

* Since we want to identify which customers are tthe most valued ones to the company, the rows with missing CustomerID are not relevant and can be dropped
* We can drop the Unnamed: 8 column, since it doesn't have any value
* We can ignore the missing values from the column Description, because the column StockCode is a reference to the same product and doesn't have any missing values

In [ ]:
df_inv.drop('Unnamed: 8', axis=1, inplace=True)
df_inv.drop(df_inv[df_inv['CustomerID'].isnull()].index, inplace=True)
df_inv.head()

In [ ]:
df_inv.info()

As we can see, all the missing values from the column Description were removed when we deleted the rows that didn't have a CustomerID value

## 1.2 New Features <a class='anchor' id='1.2'></a>

* TotalValue: Total value of the transaction 
* InvoiceYear: Year when the transaction was generated
* InvoiceMonth: Month when the transaction was generated
* InvoiceSemester: Semester when the transaction was generated

In [ ]:
df_inv['TotalValue'] = df_inv['Quantity']*df_inv['UnitPrice']
df_inv = pd.concat([df_inv, df_inv['InvoiceDate'].str.extract(r'(?P<InvoiceMonth>[A-Za-z]{3})-(?P<InvoiceYear>\d{2})')], axis=1)
df_inv['InvoiceMonth'] = pd.to_datetime(df_inv.InvoiceMonth, format='%b').dt.month
df_inv['InvoiceSemester'] = df_inv['InvoiceMonth'].apply(lambda x: 1 if x <= 6 else 2)
df_inv.head()

## 1.3 Negative Quantities <a class='anchor' id='1.3'></a>

In [ ]:
df_inv.describe()

In [ ]:
df_inv[df_inv['Quantity'] < 0].head()

In [ ]:
sorted(df_inv[df_inv['Quantity'] < 0].where(df_inv['Description'].str.startswith('D'))['Description'].dropna().unique())

In [ ]:
df_inv[df_inv['Quantity'] < 0].where(df_inv['Description'] == 'Discount')['StockCode'].dropna().unique()

As we can see there are no Description of 'Discount' misspelled and the only type of discount transactions are defined with the StockCode of 'D'

In [ ]:
len(df_inv[df_inv['Quantity'] < 0])

In [ ]:
len(df_inv[df_inv['Quantity'] < 0].where(df_inv['InvoiceNo'].str.contains('C')))

* All negative quantities transactions contains a C on it's InvoiceNo.
* We will consider these transactions as product returns and discounts (transactions described as discounts)

## 1.4 Data Analysis <a class='anchor' id='1.4'></a>

In [ ]:
def add_value_labels(ax, orientation, xspace=0, yspace=0, percentage=False):
    for rect in ax.patches:
        # Get X and Y placement of label from rect.
        if orientation == 'v':
            y_value = rect.get_height()
            x_value = rect.get_x() + rect.get_width() / 2
            label = rect.get_height()
        elif orientation == 'h':
            y_value = rect.get_y() + rect.get_height() / 4
            x_value = rect.get_width()
            label = rect.get_width()            

        # Vertical alignment for positive values
        va = 'bottom'

        if percentage==True:
            label = "{:.1%}".format(label)
        else:
            # Use Y value as label and format number with one decimal place
            label = "{:,.0f}".format(label)

        # Create annotation
        ax.annotate(
            label,                      # Use `label` as label
            (x_value, y_value),         # Place label at end of the bar
            xytext=(xspace, yspace),    # Vertically shift label by `space`
            textcoords="offset points", # Interpret `xytext` as offset in points
            ha='center',                # Horizontally center label
            va=va)                      # Vertically align label differently for
                                        # positive and negative values.

### 1.4.1 Country

* We could create an Insiders program for each country

In [ ]:
df_country = df_inv.groupby('Country').agg({'TotalValue':np.sum, 'Quantity':np.sum, 'CustomerID':'nunique', 'InvoiceNo':'nunique', 'StockCode':len}).sort_values(by='TotalValue', ascending=False)
df_country = df_country.rename(columns={'StockCode':'ProductTransactions'})
df_country

In [ ]:
print('The company has customers from {} different countries'.format(len(df_country)))

In [ ]:
plt.figure(figsize=(20,10))
ax = plt.subplot(111)
plt.title('Revenue of customers by countries')
ax.barh(df_country.index[::-1], df_country['TotalValue'][::-1])
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
ax = plt.subplot(111)
plt.title('Number of customers by countries')
ax.barh(df_country.index[::-1], df_country['CustomerID'][::-1])
plt.show()

* As we can see, the revenue and the number of customers come majorly from the United Kingdom
* There are a lot of countries with less than 10 customers and some of them contribute a lot to the total revenue
* Defining a limit percentage of eligible people in a country could leave a high value customer out of the Insiders program

### 1.4.2 Date

In [ ]:
df_month = df_inv.groupby('InvoiceMonth').agg({'TotalValue':np.sum, 'Quantity':np.sum, 'CustomerID':'nunique', 'InvoiceNo':'nunique', 'StockCode':len})
df_month = df_month.rename(columns={'StockCode':'ProductTransactions'})
df_month

In [ ]:
plt.figure(figsize=(20,10))
ax = plt.subplot(111)
plt.title('Revenue contribution by month')
ax.bar(df_month.index, df_month['TotalValue'])
add_value_labels(ax, orientation='v', yspace=5)
plt.show()

In [ ]:
df_semester = df_inv.groupby('InvoiceSemester').agg({'TotalValue':np.sum, 'Quantity':np.sum, 'CustomerID':'nunique', 'InvoiceNo':'nunique', 'StockCode':len})
df_semester = df_semester.rename(columns={'StockCode':'ProductTransactions'})
df_semester

In [ ]:
plt.figure(figsize=(20,10))
ax = plt.subplot(111)
plt.title('Revenue contribution by semester')
ax.bar(df_semester.index, df_semester['TotalValue'])
add_value_labels(ax, orientation='v', yspace=5)
plt.locator_params(axis='x', nbins=2)
plt.show()

In [ ]:
df_semester.loc[1]/df_semester.loc[2]

* The first semester of the year contributes almost 40% less than the second semester of the year in revenue
* February and April are the worst months of the year in terms of revenue contribution
* October and November are the best months of the year in terms of revenue contribution

### 1.4.3 Products

* All charts below show the 10 products that most contribute to the total revenue of the company
* Returns of these products were already included to calculate the total value of the illustrated feature

In [ ]:
df_products = df_inv.groupby('Description').agg({'TotalValue':np.sum, 'Quantity':np.sum, 'CustomerID':'nunique', 'InvoiceNo':'nunique', 'Country':'nunique', 'UnitPrice':np.mean}).sort_values(by='TotalValue', ascending=False)
df_products.head(10)

In [ ]:
print('The company has sold {} different products'.format(len(df_products)))

In [ ]:
plt.figure(figsize=(20,10))
ax = plt.subplot(111)
plt.title('Top 10 revenue contributed products')
ax.barh(df_products.index[:10][::-1], df_products['TotalValue'][:10][::-1])
plt.xlabel('Revenue')
add_value_labels(ax, orientation='h', xspace=25, yspace=5)
plt.show()

* The labels of the chart above indicate the exact amount that each product contributed

In [ ]:
plt.figure(figsize=(20,10))
ax = plt.subplot(111)
plt.title('Top 10 revenue contributed products')
ax.barh(df_products.index[:10][::-1], df_products['TotalValue'][:10][::-1]/df_products['TotalValue'].sum())
plt.xlabel('Revenue')
add_value_labels(ax, orientation='h', xspace=25, yspace=5, percentage=True)
ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.show()

In [ ]:
top_10_percentage_revenue = sum(df_products['TotalValue'][:10][::-1]/df_products['TotalValue'].sum())*100
top_10_percentage_revenue

* The labels of the chart above indicate the percentage that each product represents in the total revenue of the company
* The sum of the total revenue of these 10 products represents about 8.2% of the total revenue of the company

In [ ]:
plt.figure(figsize=(20,10))
ax = plt.subplot(111)
plt.title('Top 10 revenue contributed products')
ax.barh(df_products.index[:10][::-1], df_products['Quantity'][:10][::-1])
plt.xlabel('Quantity')
add_value_labels(ax, orientation='h', xspace=25, yspace=5)
plt.show()

* The labels of the chart above indicate the quantity of these products that were sold in total

In [ ]:
plt.figure(figsize=(20,10))
ax = plt.subplot(111)
plt.title("Top 10 revenue contributed products' unit price")
ax.barh(df_products.index[:10][::-1], df_products['UnitPrice'][:10][::-1])
plt.xlabel('Unit Price')
add_value_labels(ax, orientation='h', xspace=15, yspace=5)
plt.show()

* The labels of the chart above indicate the unit price of these products

# 2. Customers Dataframe <a class='anchor' id='2.'></a>

## 2.1 Dataframe <a class='anchor' id='2.1'></a>

* create a new dataframe grouped by CustomerID
* The columns of the new dataframe must be:
    * Country: Country of the customer
    * Revenue: Sum of all transactions total value (including negative values (returns and discounts))
    * Quantity: Sum of all transactions quantities (including negative values (returns only))
    * First Transaction: Date of the first transaction of the customer
    * Last Transaction: Date of the last transaction of the customer
    * Transactions: Total number of transactions made by each customer (include all returns and discounts)

In [ ]:
df_inv['InvoiceDate'] = pd.to_datetime(df_inv['InvoiceDate'])
df_inv['InvoiceDate2'] = pd.to_datetime(df_inv['InvoiceDate'])

df_cust = df_inv.groupby('CustomerID').agg({'Country':max, 'TotalValue':np.sum, 'Quantity':np.sum, 'InvoiceDate':min, 'InvoiceDate2':max, 'InvoiceNo':'nunique'}).sort_values(by='TotalValue', ascending=False)
df_cust = df_cust.rename(columns={'TotalValue':'Revenue', 'InvoiceDate':'First Transaction', 'InvoiceDate2':'Last Transaction', 'InvoiceNo':'Transactions'})
df_cust.head()

## 2.2 New Features <a class='anchor' id='2.2'></a>

### 2.2.1 Base Features 

* Returns: Number of return transactions
* Discounts: Number of discount transactions
* Purchases: Number of purchased product transactions

In [ ]:
df_returns = df_inv[(df_inv['StockCode'] != 'D') & (df_inv['Quantity'] < 0)].groupby('CustomerID').agg({'TotalValue':len, 'InvoiceNo':'nunique'})
df_cust['Returns'] = df_cust.index.map(df_returns['InvoiceNo'])
df_cust.head()

In [ ]:
df_discounts = df_inv[df_inv['StockCode'] == 'D'].groupby('CustomerID').agg({'TotalValue':len})
df_cust['Discounts'] = df_cust.index.map(df_discounts['TotalValue'])
df_cust.head()

Before creating the Purchases column, I will fill all the nan values with 0, to enable the substraction to occur

In [ ]:
df_cust.fillna(0, inplace=True)
df_cust['Purchases'] = df_cust['Transactions'] - df_cust['Returns'] - df_cust['Discounts']
df_cust.head()

In [ ]:
df_cust.describe()

In [ ]:
print('Number of CustomerIDs with negative TotalValue: ', len(df_cust[df_cust['Revenue'] < 0]))
print('Total number of transactions of these CustomerIDs: ', df_cust[df_cust['Revenue'] < 0]['Transactions'].sum())

* There are some CustomerIDs with negative TotalValue. These customers have some missing transactions. I will keep their transactions in the dataset, because they contribute to the total number of Customers.

### 2.2.2 Metric Features

* AOV: Average Order Value
* CLV: Customer Lifetime Value

#### 2.2.2.1 Average Order Value (AOV)

<p style='text-align:center'>$AOV = \frac{Revenue}{Number\,of\,Orders}$</p>

Where:
* Revenue: Revenue column
* Number of Orders: Transactions column

Reference: [Optimization glossary](https://www.optimizely.com/optimization-glossary/average-order-value/)

In [ ]:
df_cust['AOV'] = df_cust['Revenue']/df_cust['Purchases']
df_cust.head()

#### 2.2.2.2 Customer Lifetime Value (CLV)

<p style='text-align:center'>$CLV = Average\,Transaction\,Size\,x\,Number\,of\,Transactions\,x\,Retention\,Period$</p>

Where:
* Average Transaction Size: AOV column
* Number of Transactions: Average number of transactions per month
    * create a new column: Transactions per Month
* Retention Period: How long the average customer stick with the brand
    * create a variable: retention_period

Reference: [Oracle netsuite](https://www.netsuite.com/portal/resource/articles/ecommerce/customer-lifetime-value-clv.shtml)

In [ ]:
last_dataset_date = df_inv['InvoiceDate'].max()
last_dataset_date

In [ ]:
df_cust['Transaction Period'] = ((last_dataset_date - df_cust['First Transaction'])/np.timedelta64(1, 'M'))
df_cust['Transaction Period'] = df_cust['Transaction Period'].apply(lambda x: x if x > 1 else 1)
df_cust['Transactions per Month'] = df_cust['Purchases']/df_cust['Transaction Period']
df_cust.head()

* To create the 'Transactions per Month' column, I created the 'Transaction Period' column
* Transaction Period: result, in months, of the substraction of the final date of the dataset and each respective first transaction
* For rows with 'Transaction Period' < 0, I considered the value of 1 for this column
* Transaction per Month: divided the total number of purchases of each customer by the 'Transaction Period'. This way we can find out the average number of purchases transactions per month of each customer.

In [ ]:
retention_period = np.mean(df_cust['Last Transaction'] - df_cust['First Transaction'])/np.timedelta64(1,'M')
retention_period

The retention period was calculated by the average value, in months, of the difference between the last transaction and the first transaction of each customer.

In [ ]:
df_cust = df_cust.replace([np.inf, -np.inf], 0)
df_cust['CLV'] = df_cust['AOV']*df_cust['Transactions per Month']*retention_period
df_cust.head()

## 2.3 Data Analysis <a class='anchor' id='2.3'></a>

In [ ]:
df_cust.describe()

### 2.3.1 Revenue

In [ ]:
df_cust.head(10)

In [ ]:
plt.figure(figsize=(20,10))
ax = plt.subplot(111)
plt.title('Top 10 revenue by customer')
ax.bar(df_cust.index[:10].astype(int).astype(str), df_cust['Revenue'].iloc[:10])
plt.ylabel('Revenue')
plt.xlabel('Customer ID')
add_value_labels(ax, orientation='v', yspace=5)
plt.show()

* The chart above shows the 10 customers that most contributes to the company's revenue

In [ ]:
plt.figure(figsize=(20,10))
ax = plt.subplot(111)
plt.title('Top 10 revenue percentage by customer')
ax.bar(df_cust.index[:10].astype(int).astype(str), df_cust['Revenue'].iloc[:10]/df_cust['Revenue'].sum())
plt.ylabel('Revenue')
plt.xlabel('Customer ID')
add_value_labels(ax, orientation='v', yspace=5, percentage=True)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.show()

In [ ]:
top_10_customers_revenue = sum(df_cust['Revenue'].iloc[:10]/df_cust['Revenue'].sum())*100
top_10_customers_revenue

* Some of the customers that most contributes to the revenue are from countries with small number of customers (e.g. Netherlands and EIRE)
* The top 10 customers contribute about to 16.5% of the company's total revenue

### 2.3.2 Quantities

In [ ]:
df_cust_quantities = df_cust.sort_values(by='Quantity', ascending=False)
df_cust_quantities.head(10)

In [ ]:
plt.figure(figsize=(20,10))
ax = plt.subplot(111)
plt.title('Top 10 quantities by customer')
ax.bar(df_cust_quantities.index[:10].astype(int).astype(str), df_cust_quantities['Quantity'].iloc[:10])
plt.ylabel('Quantity')
plt.xlabel('Customer ID')
add_value_labels(ax, orientation='v', yspace=5)
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
ax = plt.subplot(111)
plt.title('Top 10 quantities by customer')
ax.bar(df_cust_quantities.index[:10].astype(int).astype(str), df_cust_quantities['Quantity'].iloc[:10]/df_cust_quantities['Quantity'].sum())
plt.ylabel('Quantity')
plt.xlabel('Customer ID')
add_value_labels(ax, orientation='v', yspace=5, percentage=True)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.show()

In [ ]:
top_10_customers_quantity = sum(df_cust['Quantity'].iloc[:10]/df_cust['Quantity'].sum())*100
top_10_customers_quantity

* Some of the customers that most contributes to the quantity of products bought are from countries with small number of customers (e.g. Netherlands and EIRE)
* The top 10 customers contribute about to 15.4% of the company's total quantities sold

### 2.3.3 Purchases

In [ ]:
df_cust_purchases = df_cust.sort_values(by='Purchases', ascending=False)
df_cust_purchases.head(10)

In [ ]:
plt.figure(figsize=(20,10))
ax = plt.subplot(111)
plt.title('Top 10 purchases transactions by customer')
ax.bar(df_cust_purchases.index[:10].astype(int).astype(str), df_cust_purchases['Purchases'].iloc[:10])
plt.ylabel('Purchases')
plt.xlabel('Customer ID')
add_value_labels(ax, orientation='v', yspace=5)
plt.show()

### 2.3.4 AOV

In [ ]:
df_cust_aov = df_cust.sort_values(by='AOV', ascending=False)
df_cust_aov.head(10)

In [ ]:
plt.figure(figsize=(20,10))
ax = plt.subplot(111)
plt.title('Top 10 AOV by customer')
ax.bar(df_cust_aov.index[:10].astype(int).astype(str), df_cust_aov['AOV'].iloc[:10])
plt.ylabel('AOV')
plt.xlabel('Customer ID')
add_value_labels(ax, orientation='v', yspace=5)
plt.show()

* The top 10 customers by AOV metric are from very distinct countries
* Since we are going to use this metric to cluster the customers and we don't want to exclude any of them only using the country as a condition, we are not going to use the country as a feature

### 2.3.5 CLV

In [ ]:
df_cust_clv = df_cust.sort_values(by='CLV', ascending=False)
df_cust_clv.head(10)

In [ ]:
plt.figure(figsize=(20,10))
ax = plt.subplot(111)
plt.title('Top 10 CLV by customer')
ax.bar(df_cust_clv.index[:10].astype(int).astype(str), df_cust_clv['CLV'].iloc[:10])
plt.ylabel('Purchases')
plt.xlabel('Customer ID')
add_value_labels(ax, orientation='v', yspace=5)
plt.show()

## 2.4 Data Preprocessing <a class='anchor' id='2.4'></a>

In [ ]:
df_cust.describe()

<b>Checking the skewness of our dataset.</b>

* A normally distribuited data has a skewness close to zero.
* Skewness greather than zero means that there is more weight in the left side of the data.
* In another hand, skewness smaller than 0 means that there is more weight in the right side of the data

<p align='center'>    <img src='images/skew.png'>
</p>

In [ ]:
df_cust.skew()

In [ ]:
sns.set(style='white',font_scale=1.3, rc={'figure.figsize':(20,20)})
ax=df_cust.hist(bins=100)

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
std_scaler = StandardScaler()
data_cluster = df_cust.select_dtypes(include=numerics).copy()
data_cluster[data_cluster.columns] = std_scaler.fit_transform(data_cluster)

In [ ]:
data_cluster.describe()

In [ ]:
columns = ['Revenue', 'Purchases', 'AOV', 'CLV']
data_cluster = data_cluster[columns]

In [ ]:
pca_2 = PCA(2)
pca_2_result = pca_2.fit_transform(data_cluster)

print ('Cumulative variance explained by 2 principal components: {:.2%}'.format(np.sum(pca_2.explained_variance_ratio_)))

In [ ]:
sns.set(style='white', rc={'figure.figsize':(9,6)},font_scale=1.1)

plt.scatter(x=pca_2_result[:, 0], y=pca_2_result[:, 1], lw=0.1)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Data represented by the 2 strongest principal components',fontweight='bold')
plt.show()

In [ ]:
sns.set(style='white', rc={'figure.figsize':(9,6)},font_scale=1.1)

plt.scatter(x=pca_2_result[:, 0], y=pca_2_result[:, 1], lw=0.1, alpha=0.2)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Data represented by the 2 strongest principal components',fontweight='bold')
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 4))
plt.imshow(pca_2.components_, interpolation = 'none', cmap = 'plasma')
feature_names = list(data_cluster.columns)

plt.gca().set_xticks(np.arange(-.5, len(feature_names)-.5));
plt.gca().set_yticks(np.arange(0.5, 2));
plt.gca().set_xticklabels(feature_names, rotation=90, ha='left', fontsize=12);
plt.gca().set_yticklabels(['First PC', 'Second PC'], va='bottom', fontsize=12);

plt.colorbar(orientation='horizontal', ticks=[pca_2.components_.min(), 0, 
                                              pca_2.components_.max()], pad=0.65);

# 3. Model <a class='anchor' id='3.'></a>

## 3.1 K-Means <a class='anchor' id='3.1'></a>
In order to define the number of clusters (K), we will combine the Silhouette Score and Inertia (Elbow plot analysis) techniques

In [ ]:
inertia = []
for i in tqdm(range(2,10)):
    kmeans = cluster.KMeans(n_clusters=i,
               init='k-means++',
               n_init=15,
               max_iter=500)
    kmeans.fit(data_cluster)
    inertia.append(kmeans.inertia_)

In [ ]:
silhouette = {}
for i in tqdm(range(2,10)):
    kmeans = cluster.KMeans(n_clusters=i,
               init='k-means++',
               n_init=15,
               max_iter=500)
    kmeans.fit(data_cluster)
    silhouette[i] = silhouette_score(data_cluster, kmeans.labels_, metric='euclidean')

In [ ]:
sns.set(style='white',font_scale=1.1, rc={'figure.figsize':(12,5)})

plt.subplot(1, 2, 1)

plt.plot(range(2,len(inertia)+2), inertia, marker='o',lw=2,ms=8)
plt.xlabel('Number of clusters')
plt.title('K-means Inertia',fontweight='bold')
plt.grid(True)

plt.subplot(1, 2, 2)

plt.bar(range(len(silhouette)), list(silhouette.values()), align='center',width=0.5)
plt.xticks(range(len(silhouette)), list(silhouette.keys()))
plt.grid()
plt.title('Silhouette Score',fontweight='bold')
plt.xlabel('Number of Clusters')


plt.show()

In [ ]:
kmeans = cluster.KMeans(n_clusters=3,init='k-means++')
kmeans_labels = kmeans.fit_predict(data_cluster)

kmeans_centroids = kmeans.cluster_centers_
kmeans_centroids_pca = pca_2.transform(kmeans_centroids)

pd.Series(kmeans_labels).value_counts()

In [ ]:
sns.set(style='white', rc={'figure.figsize':(9,6)},font_scale=1.1)

plt.scatter(x=pca_2_result[:, 0], y=pca_2_result[:, 1], c=kmeans_labels, cmap='autumn')
plt.scatter(kmeans_centroids_pca[:, 0], kmeans_centroids_pca[:, 1],
            marker='x', s=169, linewidths=3,
            color='black', zorder=10,lw=3)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Clustered Data (PCA visualization)',fontweight='bold')
plt.savefig('images/kmeans')
plt.show()

## 3.2 Agglomerative Clustering <a class='anchor' id='3.2'></a>
In order to define the number of clusters (K), we will use a dendogram

In [ ]:
plt.figure(figsize=(16,10))
dendrogram(ward(data_cluster))
plt.show()

In [ ]:
agg_clust = cluster.AgglomerativeClustering(n_clusters=2)
agg_clust_labels = agg_clust.fit_predict(data_cluster)

clf = NearestCentroid()
clf.fit(data_cluster, agg_clust_labels)

agg_clust_centroids = clf.centroids_
agg_clust_centroids_pca = pca_2.transform(agg_clust_centroids)

pd.Series(agg_clust_labels).value_counts()

In [ ]:
sns.set(style='white', rc={'figure.figsize':(9,6)},font_scale=1.1)

plt.scatter(x=pca_2_result[:, 0], y=pca_2_result[:, 1], c=agg_clust_labels, cmap='autumn')
plt.scatter(agg_clust_centroids_pca[:, 0], agg_clust_centroids_pca[:, 1],
            marker='x', s=169, linewidths=3,
            color='black', zorder=10,lw=3)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Clustered Data (PCA visualization)',fontweight='bold')
plt.show()

## 3.3 DBSCAN <a class='anchor' id='3.3'></a>

In [ ]:
dbscan = cluster.DBSCAN(eps=2,min_samples=2)
dbscan_labels = dbscan.fit_predict(data_cluster)

clf = NearestCentroid()
clf.fit(data_cluster, dbscan_labels)

dbscan_centroids = clf.centroids_
dbscan_centroids_pca = pca_2.transform(dbscan_centroids)

pd.Series(dbscan_labels).value_counts()

In [ ]:
sns.set(style='white', rc={'figure.figsize':(9,6)},font_scale=1.1)

plt.scatter(x=pca_2_result[:, 0], y=pca_2_result[:, 1], c=dbscan_labels, cmap='autumn')
plt.scatter(dbscan_centroids_pca[:, 0], dbscan_centroids_pca[:, 1],
            marker='x', s=169, linewidths=3,
            color='black', zorder=10,lw=3)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Clustered Data (PCA visualization)',fontweight='bold')
plt.show()

Analysing the 3 clustering algorithms results, I decided to use the <b>K-Means</b> algorithm. The 3 clusters provided the are:
* 0: Normal customers
* 1: Highest value customers
* 2: High value customers

We will include the clusters 1 and 2 into the Insiders group

In [ ]:
df_clust = df_cust.copy()
df_clust['Cluster'] = kmeans_labels
df_clust.describe()

### Silhouette score
* It displays a measure of how close each point in a cluster is to points in the neighbouring clusters.<br>
* The higher the Silhouette Coefficients (the closer to +1), the further away the cluster’s samples are from the neighbouring clusters samples. 
* A value of 0 indicates that the sample is on or very close to the decision boundary between two neighbouring clusters. 
* Negative values, instead, indicate that those samples might have been assigned to the wrong cluster. 
* Averaging the Silhouette Coefficients, we can get to a global Silhouette Score which can be used to describe the entire population’s performance with a single value.

Reference: [Performance Metrics in Machine Learning — Part 3: Clustering](https://towardsdatascience.com/performance-metrics-in-machine-learning-part-3-clustering-d69550662dc6)

In [ ]:
print('The average silhouette score of the k-means algorithm with 3 clusters is {:.2%}'.format(silhouette[3]))

# 4. Conclusion <a class='anchor' id='4.'></a>

In [ ]:
df_insiders = df_clust[df_clust['Cluster'] != 0]
df_insiders.head()

## 4.1 Who are the people eligible to participate in the Insiders program? <a class='anchor' id='4.1'></a>

In [ ]:
df_insiders.index

* The customers eligible to participate in the Insiders program are all people from the clusters 1 and 2 (high value customers and highest value customers).
* The list of all eligible customers can be consulted above

## 4.2 How many customers will be part of the group? <a class='anchor' id='4.2|'></a>

In [ ]:
len_insiders = len(df_insiders.index)
print('The Insiders group will be formed by {} customers'.format(len_insiders))

## 4.3 What are the main characteristics of these customers? <a class='anchor' id='4.3'></a>

In [ ]:
df_insiders.describe(include='all')

In [ ]:
df_insiders_products = df_inv[df_inv['CustomerID'].isin(df_insiders.index)].groupby('Description').agg({'TotalValue':np.sum, 'Quantity':np.sum, 'InvoiceNo':'nunique', 'Country':'nunique', 'UnitPrice':np.mean}).sort_values(by='Quantity', ascending=False)
df_insiders_products.head(10)

In [ ]:
plt.figure(figsize=(20,10))
ax = plt.subplot(111)
plt.title('Top 10 products bought by the Insiders')
ax.barh(df_insiders_products.index[:10][::-1], df_insiders_products['Quantity'][:10][::-1])
plt.xlabel('Quantity')
add_value_labels(ax, orientation='h', xspace=25, yspace=5)
plt.show()

In [ ]:
df_insiders_country = df_inv[df_inv['CustomerID'].isin(df_insiders.index)].groupby('Country').agg({'TotalValue':np.sum, 'Quantity':np.sum, 'InvoiceNo':'nunique', 'Country':'nunique', 'UnitPrice':np.mean}).sort_values(by='TotalValue', ascending=False)
df_insiders_country.head()

In [ ]:
plt.figure(figsize=(25,10))
ax = plt.subplot(111)
plt.title('Insiders customers revenue by country')
ax.barh(df_insiders_country.index[::-1], df_insiders_country['TotalValue'][::-1])
plt.xlabel('Revenue')
add_value_labels(ax, orientation='h', xspace=30)
plt.show()

In [ ]:
df_insiders_products_country = df_inv[df_inv['CustomerID'].isin(df_insiders.index)].groupby(['Country','Description']).agg({'TotalValue':np.sum, 'Quantity':np.sum, 'InvoiceNo':'nunique', 'Country':'nunique', 'UnitPrice':np.mean}).sort_values(by='Quantity', ascending=False)
df_insiders_products_country.head()

In [ ]:
df_insiders_products_country.groupby(level=0).head(1)

## 4.4 What is the percentage of revenue contribution, coming from Insiders? <a class='anchor' id='4.4'></a>

In [ ]:
insiders_percentage = len(df_insiders)/len(df_clust)
revenue_percentage = df_insiders['Revenue'].sum()/df_clust['Revenue'].sum()
print('The percentage of revenue contribution of the Insiders is {:.2%}.'.format(revenue_percentage))

In [ ]:
sns.set(style='white',font_scale=1.1, rc={'figure.figsize':(12,5)})
colors = ['red', 'darkgrey']

ax = plt.subplot(1, 2, 1)
sizes = [insiders_percentage, 1-insiders_percentage]
plt.title('All In One Place customers', fontweight='bold')
_, _, autopcts = ax.pie(sizes, autopct='%1.1f%%', pctdistance=1.25, colors=colors)
plt.setp(autopcts, **{'fontsize':15, 'fontweight': 'bold'})

ax = plt.subplot(1, 2, 2)
sizes = [revenue_percentage, 1-revenue_percentage]
plt.title('All In One Place revenue', fontweight='bold')
_, _, autopcts = ax.pie(sizes, autopct='%1.1f%%', colors=colors)
plt.setp(autopcts, **{'fontsize':15, 'fontweight': 'bold'})


legend = ['Insiders', 'Normal Customers']
plt.legend(legend, loc=3, bbox_to_anchor=(-0.4,0), framealpha=0.0, prop={'size': 12})

plt.show()

## 4.5 What is this group's revenue expectation for the coming months? <a class='anchor' id='4.5'></a>

In [ ]:
aov = df_insiders.describe()['AOV'].loc['mean']
transactions_per_month = df_insiders.describe()['Transactions per Month'].loc['mean']
individual_revenue_expectation = aov*transactions_per_month
total_revenue_expectation = individual_revenue_expectation * len_insiders
print("The Insiders group revenue expectation for the next months is {:,.2f} per month".format(total_revenue_expectation))

## 4.6 What are the conditions for a person to be eligible for Insiders? <a class='anchor' id='4.6'></a>

* After computing the person data (Revenue, Number of Purchase Orders, AOV and CLV), the system will evaluate to what cluster this customer belongs.
* In general, after the PCA transformation needs to have a smaller euclidean distance from any of the centroids from the clusters 1 and 2 in comparison to the euclidean distance to the centroid of cluster 0.

In [ ]:
new_customer = np.array([20,0])
new_customer

sns.set(style='white', rc={'figure.figsize':(9,6)},font_scale=1.1)

plt.scatter(x=pca_2_result[:, 0], y=pca_2_result[:, 1], c=kmeans_labels, cmap='autumn')
plt.scatter(kmeans_centroids_pca[:, 0], kmeans_centroids_pca[:, 1],
            marker='x', s=169, linewidths=3,
            color='black', zorder=10,lw=3)
plt.scatter(new_customer[0], new_customer[1], marker='x', s=169, linewidths=3,
            color='blue', zorder=10,lw=3)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Clustered Data (PCA visualization)',fontweight='bold')
plt.show()

i=0
for center in kmeans_centroids_pca:
    print('Euclidean distance o centroid {}: {:.2f}'.format(i,distance.euclidean(center, new_customer)))
    i+=1

* In the example above, the customer (marked by the blue X) would be classified in the cluster 1 (Highest value customers), because the lowest euclidean distance found was from centroid 1.
* One condition to be eligible for the Insiders program is to maintain classified in cluster 1 or 2 for at least 6 months.

## 4.7 What are the conditions for a person to be removed from Insiders? <a class='anchor' id='4.7'></a>

* To be removed from the Insiders group, a customer needs to be classified in cluster 0 (Normal customers) for at least 3 months in a row.
* Another way is: every 365 days, the data of a customer in this period will be used to classify him/her into the 3 clusters. If the customer is classified as a normal customer in this one year interval, he/she will be removed from the Insiders program.

## 4.8 What is the guarantee that the Insiders program is better than the rest of the base? <a class='anchor' id='4.8'></a>

In [ ]:
# Normal customers
df_clust[df_clust['Cluster'] == 0].describe().loc['mean']

In [ ]:
# Insiders
df_insiders.describe().loc['mean']

In [ ]:
df_clust.corr()['Cluster'].sort_values(ascending=False)

* As we can compare, all relevant mean values are way higher in the Insiders group.
* Since new customers that becomes eligible for the program will eventually be part of the program and customers that doesn't keep their averages enough will be removed, we guarantee that the Insiders will always be better than the rest.

## 4.9 What actions can the marketing team take to increase revenue? <a class='anchor' id='4.9'></a>

1. Create a survey to have some feedbacks from customers and take actions to improve in these lack points

Consider asking the following questions: 

* How do you go about searching for and selecting products on our site? 
* Is there any product you searched for but couldn’t find? 
* Which aspects of our site/products do you like most? 
* Which improvements in your life or business have you seen after using our site? 
* If there was one thing we could do better what would it be? 

2. Create some personalizations for the customers from different countries

We know by now that personalization can deliver five to eight times the ROI on marketing spend, and can lift sales by 10% or more. With the help of omnichannel personalization engines, marketers can leverage prior behavioral data and apply micro-segmentation as well as machine learning algorithms to personalize the buying experiences of their most valuable customers.

For example, a large U.S. fashion retailer with six major sub-brands encouraged visitors to browse through more products by converting every homepage component into a dynamic element that targets different audience segments.

Reference: [Targeting your most valuable customers in a post-Pareto world](https://www.dynamicyield.com/lesson/targeting-high-value-customers/)